In [ ]:
%pip install qiskit==1.2.4
%pip install qiskit-aer==0.15.1
%pip install pylatexenc==2.10

In [285]:
from qiskit import QuantumCircuit
from qiskit.converters import circuit_to_gate
from qiskit.visualization import array_to_latex
from qiskit.quantum_info import Operator
from qiskit.quantum_info import Statevector
from qiskit import transpile 
from qiskit.providers.basic_provider import BasicSimulator
from qiskit.visualization import plot_histogram
from qiskit.circuit import ControlledGate
import math 

# The aim of the assignment is to simulate the Ekert91 key distribution protocol.

# This notebook is for a simulation of the protocol with an attacker, to demonstrate that the attacker can be detected.

choosingMatrix = [[1/math.sqrt(3), -math.sqrt(2/3)],[math.sqrt(2/3), 1/math.sqrt(3)]]

wTransformMatrix = [[1/math.sqrt(2), 1/math.sqrt(2)],[1/math.sqrt(2), -1/math.sqrt(2)]]
vTransformMatrix = [[-1/math.sqrt(2), 1/math.sqrt(2)],[1/math.sqrt(2), 1/math.sqrt(2)]]

def entangledPair():
    q = QuantumCircuit(2)
    q.x(1)
    q.h(0)
    q.cx(0,1)
    q.z(0)
    return q

def basis_choice():
    #returns a number 1 - 3 that decides which i/j A and B will use
    choosingCircuit = QuantumCircuit(1)
    choosingCircuit.unitary(choosingMatrix, [0])
    choosingCircuit.measure_all()
    backend = BasicSimulator()
    compiled = transpile(choosingCircuit, backend)
    job_sim = backend.run(compiled, shots=1)
    result = job_sim.result()
    counts = result.get_counts()
    if(counts == {'0': 1}):
        return 1
    elif(counts == {'1': 1}):
        secondaryChoice = QuantumCircuit(1)
        secondaryChoice.h(0)
        secondaryChoice.measure_all()
        backend = BasicSimulator()
        compiled = transpile(secondaryChoice, backend)
        job_sim = backend.run(compiled, shots=1)
        result = job_sim.result()
        counts = result.get_counts()
        if(counts == {'0': 1}):
            return 2
        elif(counts == {'1': 1}):
            return 3

def measure_alice(pair, basis):
    if basis == 1:
        #print("AX")
        pair.h(0)
        return pair
    if basis == 2:
        #print("AW")
        pair.unitary(wTransformMatrix, [0])
        return pair
    if basis == 3:
        #print("AZ")
        return pair
    
def measure_bob(pair, basis):
    if basis == 1:
        #print("BW")
        pair.unitary(wTransformMatrix, [1])
        return pair
    if basis == 2:
        #print("BZ")
        return pair
    if basis == 3:
        #print("BV")
        pair.unitary(vTransformMatrix, [1])
        return pair

def attacker(pair):
    pair.reset(1)
    randomFlip = QuantumCircuit(1)
    randomFlip.h(0)
    randomFlip.measure_all()
    backend = BasicSimulator()
    compiled = transpile(randomFlip, backend)
    job_sim = backend.run(compiled, shots=1)
    result = job_sim.result()
    counts = result.get_counts()
    if(counts == {'0': 1}):
        pass
    elif(counts == {'1': 1}):
        pair.x(1)
    return pair

def conv(qubit):
    return 1 if qubit == 0 else -1

In [ ]:
#generation of the bases and results
n = 4
attack = True
rounds = int((9*n)/2)
if(not isinstance(rounds, int)):
    print("rounds must be an integer")

aliceBases = []
bobBases = []
aliceResults = []
bobResults = []

for i in range(rounds):
    pair = entangledPair()

    alicechoice = basis_choice()
    aliceBases.append(alicechoice)

    bobchoice = basis_choice()
    bobBases.append(bobchoice)
    
    pair = measure_alice(pair, alicechoice)
    if(attack):
        pair = attacker(pair)
    pair = measure_bob(pair, bobchoice)
    pair.measure_all()

    backend = BasicSimulator()
    compiled = transpile(pair, backend)
    job_sim = backend.run(compiled, shots=1)
    result = job_sim.result()
    counts = result.get_counts()
    if(counts == {'00': 1}):
        aliceResults.append(0)
        bobResults.append(0)
    elif(counts == {'01': 1}):
        aliceResults.append(0)
        bobResults.append(1)
    elif(counts == {'10': 1}):
        aliceResults.append(1)
        bobResults.append(0)
    elif(counts == {'11': 1}):
        aliceResults.append(1)
        bobResults.append(1)


print("Alice bases:   %s" % aliceBases)
print("Bob bases:     %s" % bobBases) 
print("Alice results: %s" % aliceResults)
print("Bob results:   %s" % bobResults)

In [ ]:
#


xwSum = 0
xwCount = 0
xvSum = 0
xvCount = 0
zwSum = 0
zwCount = 0
zvSum = 0
zvCount = 0

for i in range(len(aliceBases)):
    if((aliceBases[i] == 1 or aliceBases[i] == 3) and (bobBases[i] == 1 or bobBases[i] == 3)):
        a = conv(aliceResults[i])
        b = conv(bobResults[i])
        if(aliceBases[i] == 1):
            if(bobBases[i] == 1):
                xwSum += a * b
                xwCount += 1
            elif(bobBases[i] == 3):
                xvSum += a * b
                xvCount += 1
        elif(aliceBases[i] == 3):
            if(bobBases[i] == 1):
                zwSum += a * b
                zwCount += 1
            elif(bobBases[i] == 3):
                zvSum += a * b
                zvCount += 1

if(xwCount != 0):
    avgXW = xwSum / xwCount
else:
    avgXW = 0
if(xvCount != 0):
    avgXV = xvSum / xvCount
else:
    avgXV = 0
if(zwCount != 0):
    avgZW = zwSum / zwCount
else:
    avgZW = 0
if(zvCount != 0):
    avgZV = zvSum / zvCount
else:
    avgZV = 0

S = abs(avgXW - avgXV + avgZW + avgZV)

if(S < 2):
    print("Attacker detected, no key generated, bell inequality value: %f" % S)
else:
    key = []
    for i in range(len(aliceBases)):
        if((aliceBases[i] == 2 and bobBases[i] == 1) or (aliceBases[i] == 3 and bobBases[i] == 2)):
            if(aliceResults[i] != bobResults[i]):
                bobResults[i] = 1 - bobResults[i]
                key.append(aliceResults[i])
    print("Shared key (length %i): %s" % (len(key), key))
    print("Bell inequality value: %f" % S)